## Using BQML

In [33]:
%load_ext google.cloud.bigquery

In [7]:
%%bigquery

#standardSQL
WITH visitors AS(
SELECT
COUNT(DISTINCT fullVisitorId) AS total_visitors
FROM `data-to-insights.ecommerce.web_analytics`
),
purchasers AS(
SELECT
COUNT(DISTINCT fullVisitorId) AS total_purchasers
FROM `data-to-insights.ecommerce.web_analytics`
WHERE totals.transactions IS NOT NULL
)
SELECT
  total_visitors,
  total_purchasers,
  total_purchasers / total_visitors AS conversion_rate
FROM visitors, purchasers

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.20s/rows]


,total_visitors,total_purchasers,conversion_rate
0,741721,20015,0.026985


### What's the top 5 selling products?

In [8]:
%%bigquery

SELECT
  p.v2ProductName,
  p.v2ProductCategory,
  SUM(p.productQuantity) AS units_sold,
  ROUND(SUM(p.localProductRevenue/1000000),2) AS revenue
FROM `data-to-insights.ecommerce.web_analytics`,
UNNEST(hits) AS h,
UNNEST(h.product) AS p
GROUP BY 1, 2
ORDER BY revenue DESC
LIMIT 5;

Downloading: 100%|██████████| 5/5 [00:01<00:00,  4.03rows/s]


,v2ProductName,v2ProductCategory,units_sold,revenue
0,Nest® Learning Thermostat 3rd Gen-USA - Stainl...,Nest-USA,17651,870976.95
1,Nest® Cam Outdoor Security Camera - USA,Nest-USA,16930,684034.55
2,Nest® Cam Indoor Security Camera - USA,Nest-USA,14155,548104.47
3,Nest® Protect Smoke + CO White Wired Alarm-USA,Nest-USA,6394,178937.60
4,Nest® Protect Smoke + CO White Battery Alarm-USA,Nest-USA,6340,178572.40


### How many visitors bought on subsequent visits to the website?

In [23]:
%%bigquery

WITH all_visitor_stats AS (
SELECT
  fullvisitorid, # 741,721 unique visitors
  IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid
)
SELECT
  COUNT(DISTINCT fullvisitorid) AS total_visitors,
  will_buy_on_return_visit
FROM all_visitor_stats
GROUP BY will_buy_on_return_visit

Downloading: 100%|██████████| 2/2 [00:01<00:00,  1.64rows/s]


,total_visitors,will_buy_on_return_visit
0,729848,0
1,11873,1


### Select features and create training

In [24]:
%%bigquery

SELECT
  * EXCEPT(fullVisitorId)
FROM
  # features
  (SELECT
    fullVisitorId,
    IFNULL(totals.bounces, 0) AS bounces,
    IFNULL(totals.timeOnSite, 0) AS time_on_site
  FROM
    `data-to-insights.ecommerce.web_analytics`
  WHERE
    totals.newVisits = 1)
  JOIN
  (SELECT
    fullvisitorid,
    IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM
      `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid)
  USING (fullVisitorId)
ORDER BY time_on_site DESC
LIMIT 10;

Downloading: 100%|██████████| 10/10 [00:01<00:00,  7.84rows/s]


,bounces,time_on_site,will_buy_on_return_visit
0,0,15047,0
1,0,12136,0
2,0,11201,0
3,0,10046,0
4,0,9974,0
5,0,9564,0
6,0,9520,0
7,0,9275,1
8,0,9138,0
9,0,8872,0


### Create a BigQuery Dataset

In [36]:
DATASET='bqml_demos'

In [37]:
!bq --location='US' mk -d {DATASET}

Dataset 'jchavezar-demo:bqml_demos' successfully created.


#### We'll be using logistic regression since we need to find if a visitor will buy or not in the future

In [40]:
%%bigquery

CREATE OR REPLACE MODEL `bqml_demos.classification_model`
OPTIONS
(
model_type='logistic_reg',
labels = ['will_buy_on_return_visit']
)
AS
#standardSQL
SELECT
  * EXCEPT(fullVisitorId)
FROM
  # features
  (SELECT
    fullVisitorId,
    IFNULL(totals.bounces, 0) AS bounces,
    IFNULL(totals.timeOnSite, 0) AS time_on_site
  FROM
    `data-to-insights.ecommerce.web_analytics`
  WHERE
    totals.newVisits = 1
    AND date BETWEEN '20160801' AND '20170430') # train on first 9 months
  JOIN
  (SELECT
    fullvisitorid,
    IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM
      `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid)
  USING (fullVisitorId)
;

Query complete after 0.01s: 100%|██████████| 3/3 [00:00<00:00, 605.41query/s] 


""


### Evaluation

In [42]:
from google.cloud import bigquery

client = bigquery.Client()
sql = """
SELECT
  roc_auc,
  CASE
    WHEN roc_auc > .9 THEN 'good'
    WHEN roc_auc > .8 THEN 'fair'
    WHEN roc_auc > .7 THEN 'decent'
    WHEN roc_auc > .6 THEN 'not great'
  ELSE 'poor' END AS model_quality
FROM
  ML.EVALUATE(MODEL bqml_demos.classification_model,  (
SELECT
  * EXCEPT(fullVisitorId)
FROM
  # features
  (SELECT
    fullVisitorId,
    IFNULL(totals.bounces, 0) AS bounces,
    IFNULL(totals.timeOnSite, 0) AS time_on_site
  FROM
    `data-to-insights.ecommerce.web_analytics`
  WHERE
    totals.newVisits = 1
    AND date BETWEEN '20170501' AND '20170630') # eval on 2 months
  JOIN
  (SELECT
    fullvisitorid,
    IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM
      `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid)
  USING (fullVisitorId)
));
"""

df = client.query(sql).to_dataframe()

In [43]:
df.head()

,roc_auc,model_quality
0,0.723826,decent


![](images/vertex-bqml-roc.png)

### Improving Model Performance

In [51]:
%%bigquery

CREATE OR REPLACE MODEL `bqml_demos.classification_model_2`
OPTIONS
  (model_type='logistic_reg', labels = ['will_buy_on_return_visit']) AS
WITH all_visitor_stats AS (
SELECT
  fullvisitorid,
  IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid
)
# add in new features
SELECT * EXCEPT(unique_session_id) FROM (
  SELECT
      CONCAT(fullvisitorid, CAST(visitId AS STRING)) AS unique_session_id,
      # labels
      will_buy_on_return_visit,
      MAX(CAST(h.eCommerceAction.action_type AS INT64)) AS latest_ecommerce_progress,
      # behavior on the site
      IFNULL(totals.bounces, 0) AS bounces,
      IFNULL(totals.timeOnSite, 0) AS time_on_site,
      IFNULL(totals.pageviews, 0) AS pageviews,
      # where the visitor came from
      trafficSource.source,
      trafficSource.medium,
      channelGrouping,
      # mobile or desktop
      device.deviceCategory,
      # geographic
      IFNULL(geoNetwork.country, "") AS country
  FROM `data-to-insights.ecommerce.web_analytics`,
     UNNEST(hits) AS h
    JOIN all_visitor_stats USING(fullvisitorid)
  WHERE 1=1
    # only predict for new visits
    AND totals.newVisits = 1
    AND date BETWEEN '20160801' AND '20170430' # train 9 months
  GROUP BY
  unique_session_id,
  will_buy_on_return_visit,
  bounces,
  time_on_site,
  totals.pageviews,
  trafficSource.source,
  trafficSource.medium,
  channelGrouping,
  device.deviceCategory,
  country
);

Query complete after 0.01s: 100%|██████████| 3/3 [00:00<00:00, 539.85query/s]


""


### Evaluate Improved Model

In [54]:
%%bigquery

#standardSQL
SELECT
  roc_auc,
  CASE
    WHEN roc_auc > .9 THEN 'good'
    WHEN roc_auc > .8 THEN 'fair'
    WHEN roc_auc > .7 THEN 'decent'
    WHEN roc_auc > .6 THEN 'not great'
  ELSE 'poor' END AS model_quality
FROM
  ML.EVALUATE(MODEL bqml_demos.classification_model_2,  (
WITH all_visitor_stats AS (
SELECT
  fullvisitorid,
  IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid
)
# add in new features
SELECT * EXCEPT(unique_session_id) FROM (
  SELECT
      CONCAT(fullvisitorid, CAST(visitId AS STRING)) AS unique_session_id,
      # labels
      will_buy_on_return_visit,
      MAX(CAST(h.eCommerceAction.action_type AS INT64)) AS latest_ecommerce_progress,
      # behavior on the site
      IFNULL(totals.bounces, 0) AS bounces,
      IFNULL(totals.timeOnSite, 0) AS time_on_site,
      totals.pageviews,
      # where the visitor came from
      trafficSource.source,
      trafficSource.medium,
      channelGrouping,
      # mobile or desktop
      device.deviceCategory,
      # geographic
      IFNULL(geoNetwork.country, "") AS country
  FROM `data-to-insights.ecommerce.web_analytics`,
     UNNEST(hits) AS h
    JOIN all_visitor_stats USING(fullvisitorid)
  WHERE 1=1
    # only predict for new visits
    AND totals.newVisits = 1
    AND date BETWEEN '20170501' AND '20170630' # eval 2 months
  GROUP BY
  unique_session_id,
  will_buy_on_return_visit,
  bounces,
  time_on_site,
  totals.pageviews,
  trafficSource.source,
  trafficSource.medium,
  channelGrouping,
  device.deviceCategory,
  country
)
));

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.23s/rows]


,roc_auc,model_quality
0,0.90947,good


### Predict which new visitors will come back

In [1]:
from google.cloud import bigquery

client = bigquery.Client()

sql="""SELECT
*
FROM
  ml.PREDICT(MODEL `bqml_demos.classification_model_2`,
   (
WITH all_visitor_stats AS (
SELECT
  fullvisitorid,
  IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid
)
  SELECT
      CONCAT(fullvisitorid, '-',CAST(visitId AS STRING)) AS unique_session_id,
      # labels
      will_buy_on_return_visit,
      MAX(CAST(h.eCommerceAction.action_type AS INT64)) AS latest_ecommerce_progress,
      # behavior on the site
      IFNULL(totals.bounces, 0) AS bounces,
      IFNULL(totals.timeOnSite, 0) AS time_on_site,
      totals.pageviews,
      # where the visitor came from
      trafficSource.source,
      trafficSource.medium,
      channelGrouping,
      # mobile or desktop
      device.deviceCategory,
      # geographic
      IFNULL(geoNetwork.country, "") AS country
  FROM `data-to-insights.ecommerce.web_analytics`,
     UNNEST(hits) AS h
    JOIN all_visitor_stats USING(fullvisitorid)
  WHERE
    # only predict for new visits
    totals.newVisits = 1
    AND date BETWEEN '20170701' AND '20170801' # test 1 month
  GROUP BY
  unique_session_id,
  will_buy_on_return_visit,
  bounces,
  time_on_site,
  totals.pageviews,
  trafficSource.source,
  trafficSource.medium,
  channelGrouping,
  device.deviceCategory,
  country
)
)
ORDER BY
  predicted_will_buy_on_return_visit DESC;"""


df_2 = client.query(sql).to_dataframe()

In [9]:
df_2[['predicted_will_buy_on_return_visit', 'will_buy_on_return_visit']]

,predicted_will_buy_on_return_visit,will_buy_on_return_visit
0,1,0
1,1,0
2,1,1
3,1,0
4,1,0
...,...,...
59597,0,0
59598,0,0
59599,0,0
59600,0,0


### Explainable AI

In [34]:
%%bigquery

SELECT
  *
FROM
  ML.EXPLAIN_PREDICT(MODEL `jchavezar-demo.bqml_demos.classification_model_2`,
    (
    WITH all_visitor_stats AS (
SELECT
  fullvisitorid,
  IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid
)
  SELECT
      CONCAT(fullvisitorid, '-',CAST(visitId AS STRING)) AS unique_session_id,
      # labels
      will_buy_on_return_visit,
      MAX(CAST(h.eCommerceAction.action_type AS INT64)) AS latest_ecommerce_progress,
      # behavior on the site
      IFNULL(totals.bounces, 0) AS bounces,
      IFNULL(totals.timeOnSite, 0) AS time_on_site,
      totals.pageviews,
      # where the visitor came from
      trafficSource.source,
      trafficSource.medium,
      channelGrouping,
      # mobile or desktop
      device.deviceCategory,
      # geographic
      IFNULL(geoNetwork.country, "") AS country
  FROM `data-to-insights.ecommerce.web_analytics`,
     UNNEST(hits) AS h
    JOIN all_visitor_stats USING(fullvisitorid)
  WHERE
    # only predict for new visits
    totals.newVisits = 1
    AND date BETWEEN '20170701' AND '20170801' # test 1 month
  GROUP BY
  unique_session_id,
  will_buy_on_return_visit,
  bounces,
  time_on_site,
  totals.pageviews,
  trafficSource.source,
  trafficSource.medium,
  channelGrouping,
  device.deviceCategory,
  country))

Downloading:   0%|          | 0/59602 [00:00<?, ?rows/s]

Downloading: 100%|██████████| 59602/59602 [00:01<00:00, 45333.81rows/s]


,predicted_will_buy_on_return_visit,probability,top_feature_attributions,baseline_prediction_value,prediction_value,approximation_error,unique_session_id,will_buy_on_return_visit,latest_ecommerce_progress,bounces,time_on_site,pageviews,source,medium,channelGrouping,deviceCategory,country
0,0,0.922973,"[{'feature': 'latest_ecommerce_progress', 'att...",-0.787551,-2.483439,0.0,4161728169885185639-1500843594,0,6,0,2378,48,google,organic,Organic Search,desktop,Germany
1,0,0.985542,"[{'feature': 'country', 'attribution': -1.0275...",-0.787551,-4.221956,0.0,9449547852331358002-1500379758,0,3,0,331,8,google,organic,Organic Search,desktop,Germany
2,0,0.993963,"[{'feature': 'deviceCategory', 'attribution': ...",-0.787551,-5.103726,0.0,6894071580463767601-1499885617,0,0,0,451,26,google,organic,Organic Search,tablet,United Kingdom
3,0,0.992452,"[{'feature': 'deviceCategory', 'attribution': ...",-0.787551,-4.878872,0.0,0815583452398462354-1500131659,0,0,0,2067,20,google,organic,Organic Search,mobile,Mexico
4,0,0.985025,"[{'feature': 'country', 'attribution': -1.0237...",-0.787551,-4.186294,0.0,3499346520725035965-1499334996,0,3,0,146,13,google,organic,Organic Search,desktop,Netherlands
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59597,1,0.618164,"[{'feature': 'latest_ecommerce_progress', 'att...",-0.787551,0.481761,0.0,7284253778393190956-1500306997,0,6,0,888,28,sites.google.com,referral,Referral,desktop,United States
59598,1,0.621084,"[{'feature': 'latest_ecommerce_progress', 'att...",-0.787551,0.494150,0.0,7961103453479093572-1501174706,0,6,0,1099,26,sites.google.com,referral,Referral,desktop,United States
59599,1,0.522651,"[{'feature': 'latest_ecommerce_progress', 'att...",-0.787551,0.090666,0.0,4129015306190303528-1501524039,0,6,0,312,15,gdeals.googleplex.com,referral,Referral,desktop,United States
59600,1,0.570803,"[{'feature': 'latest_ecommerce_progress', 'att...",-0.787551,0.285127,0.0,0920738466459025144-1499651436,0,6,0,510,27,gdeals.googleplex.com,referral,Referral,desktop,United States


In [38]:
import numpy as np

df_2['diff'] = np.where(df_2['predicted_will_buy_on_return_visit'] == df_2['will_buy_on_return_visit'], 'True', 'False')
print(len(df_2[df_2['diff'].values == 'True'])/len(df_2))

0.9849501694574008


In [39]:
df_2[df_2['diff'].values == 'True']

,predicted_will_buy_on_return_visit,predicted_will_buy_on_return_visit_probs,unique_session_id,will_buy_on_return_visit,latest_ecommerce_progress,bounces,time_on_site,pageviews,source,medium,channelGrouping,deviceCategory,country,diff
2,1,"[{'label': 1, 'prob': 0.5807093147444036}, {'l...",4766623641504040239-1500335893,1,6,0,1305,19,gdeals.googleplex.com,referral,Referral,desktop,United States,True
14,1,"[{'label': 1, 'prob': 0.5256733932846221}, {'l...",5713365943321772776-1499303248,1,6,0,378,15,gdeals.googleplex.com,referral,Referral,desktop,United States,True
19,1,"[{'label': 1, 'prob': 0.5254823923358961}, {'l...",8015915032010696677-1500581774,1,5,0,1198,16,sites.google.com,referral,Referral,desktop,United States,True
20,1,"[{'label': 1, 'prob': 0.5586449908880936}, {'l...",9756202106186308060-1499477518,1,6,0,337,17,sites.google.com,referral,Referral,desktop,United States,True
32,1,"[{'label': 1, 'prob': 0.5350801524568158}, {'l...",4216294448475777248-1499048804,1,3,0,351,12,moma.corp.google.com,referral,Referral,desktop,United States,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59597,0,"[{'label': 1, 'prob': 0.0052878665897339505}, ...",2577005515154248957-1501177138,0,0,0,121,3,youtube.com,referral,Social,desktop,Romania,True
59598,0,"[{'label': 1, 'prob': 0.007090356957853872}, {...",7582261539527099659-1500270143,0,0,0,85,3,m.facebook.com,referral,Social,mobile,United States,True
59599,0,"[{'label': 1, 'prob': 0.009842112431380279}, {...",1671504134842863799-1500187178,0,0,0,72,3,youtube.com,referral,Social,desktop,United States,True
59600,0,"[{'label': 1, 'prob': 0.005286459217437018}, {...",1516264646025877273-1500100000,0,0,0,117,3,youtube.com,referral,Social,desktop,India,True


## Using TF-KERAS

### Constants

In [42]:
TRAIN_IMAGE_URI = "gcr.io/jchavezar-demo/visitor-purchase-train:latest"
STAGING_BUCKET = "gs://vtx-staging"

### Create Folder Structure

In [49]:
!rm -fr custom_7
!mkdir custom_7

### Create the Training Code

- I used 2 components here: 
    - **Bigquery**; to rapidly create my dataset from analysis and transformations in SQL.
    - **Tensorflow-Keras**; to create the ML Neural Network layers

In [ ]:
%%writefile custom_7/main.py
import os
import argparse
import numpy as np
import pandas as pd
import tensorflow as tf
from google.cloud import bigquery 
from tensorflow.keras import layers
import warnings
warnings.filterwarnings('ignore')

### Bigquery Definition

query = """
WITH all_visitor_stats AS (
SELECT
  fullvisitorid,
  IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid
)
# add in new features
SELECT * EXCEPT(unique_session_id) FROM (
  SELECT
      CONCAT(fullvisitorid, CAST(visitId AS STRING)) AS unique_session_id,
      # labels
      will_buy_on_return_visit,
      MAX(CAST(h.eCommerceAction.action_type AS INT64)) AS latest_ecommerce_progress,
      # behavior on the site
      IFNULL(totals.bounces, 0) AS bounces,
      IFNULL(totals.timeOnSite, 0) AS time_on_site,
      IFNULL(totals.pageviews, 0) AS pageviews,
      # where the visitor came from
      trafficSource.source,
      trafficSource.medium,
      channelGrouping,
      # mobile or desktop
      device.deviceCategory,
      # geographic
      IFNULL(geoNetwork.country, "") AS country
  FROM `data-to-insights.ecommerce.web_analytics`,
     UNNEST(hits) AS h
    JOIN all_visitor_stats USING(fullvisitorid)
  WHERE 1=1
    # only predict for new visits
    AND totals.newVisits = 1
    AND date BETWEEN '20160801' AND '20170430' # train 9 months
  GROUP BY
  unique_session_id,
  will_buy_on_return_visit,
  bounces,
  time_on_site,
  totals.pageviews,
  trafficSource.source,
  trafficSource.medium,
  channelGrouping,
  device.deviceCategory,
  country)
"""

def get_args():
    '''Parses args. Must include all hyperparameters you want to tune.'''
    
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--batch_size',
        default=256,
        type=int,
        help='batch size to build tf.data.Dataset')
    parser.add_argument(
        '--learning_rate',
        default=0.001,
        type=float,
        help='learning rate')
    parser.add_argument(
        '--num_neurons',
        default=32,
        type=int,
        help='number of units in the first hidden layer')
    parser.add_argument(
        '--label_column',
        default='will_buy_on_return_visit',
        type=str,
        help='The column to predict (label/target)')
    parser.add_argument(
        '--epochs',
        default=10,
        type=int,
        help='Numbber of epochs for the training; complete pass over dataset')
    args = parser.parse_args()
    return args

def preprocessing(batch_size: int, target_column: str):
  '''Split and Transform data into tf.Dataset, shuffles + batch'''
  
  client = bigquery.Client(project=os.environ['CLOUD_ML_PROJECT_ID'])
  df = client.query(query).to_dataframe()
  train, val, test = np.split(df.sample(frac=1), [int(0.8*len(df)), int(0.9*len(df))])


  ## Transforming data to tf.data.Dataset (Multidimmension)
  
  def df_to_dataset(dataframe, batch_size: str, shuffle=True):
    for column in dataframe.columns:
      if dataframe[column].dtype == 'Int64':
        dataframe[column] = dataframe[column].astype(np.int64)
      df = dataframe.copy()
      labels = df.pop(target_column)
      df = {key: value[:,tf.newaxis] for key, value in dataframe.items()}
      ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
      if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
      ds = ds.batch(batch_size)
      ds = ds.prefetch(batch_size)
    return ds

  ## Tensorflow Normalization

  def get_normalization_layer(name, dataset):
    
    normalizer = layers.Normalization(axis=None)
    feature_ds = dataset.map(lambda x, y: x[name])    
    normalizer.adapt(feature_ds)
    
    return normalizer

  ## Encoding for categorical data

  def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):

    if dtype == 'string':
      index = layers.StringLookup(max_tokens=max_tokens)
    else:
      index = layers.IntegerLookup(max_tokens=max_tokens)

    feature_ds = dataset.map(lambda x, y: x[name])
    index.adapt(feature_ds)
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())

    # Apply multi-hot encoding to the indices. The lambda function captures the
    # layer, so you can use them, or include them in the Keras Functional model later.
    return lambda feature: encoder(index(feature))
    
  train_ds = df_to_dataset(train, batch_size=batch_size)
  val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
  test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)
  
  all_inputs = []
  encoded_features = []

  # Numerical features.
  cat_columns = [i for i in df if df[i].dtypes == 'object' and i != target_column]
  num_columns = [i for i in df if df[i].dtypes == 'int64' and i != target_column]

  for header in num_columns:
    numeric_col = tf.keras.Input(shape=(1,), name=header)
    normalization_layer = get_normalization_layer(header, train_ds)
    encoded_numeric_col = normalization_layer(numeric_col)
    all_inputs.append(numeric_col)
    encoded_features.append(encoded_numeric_col)

  for header in cat_columns:
    categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='string')
    encoding_layer = get_category_encoding_layer(
      name=header,
      dataset=train_ds,
      dtype='string',
      max_tokens=5)
  encoded_categorical_col = encoding_layer(categorical_col)
  all_inputs.append(categorical_col)
  encoded_features.append(encoded_categorical_col)

  return all_inputs, encoded_features, train_ds, val_ds, test_ds

def create_model(all_inputs, encoded_features, nn_input: int, lr: float):
  '''Train model with TF+Keras'''
  all_features = tf.keras.layers.concatenate(encoded_features)
  x = tf.keras.layers.Dense(nn_input, activation="relu")(all_features)
  x = tf.keras.layers.Dropout(0.5)(x)
  output = tf.keras.layers.Dense(1)(x)
  
  model = tf.keras.Model(all_inputs, output)
  model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=["accuracy"])
  return model  

def main():
    args = get_args()
    all_inputs, encoded_features, train_ds, val_ds, test_ds = preprocessing(args.batch_size, args.label_column)
    model = create_model(all_inputs, encoded_features, args.num_neurons, args.learning_rate)
    model.fit(train_ds, epochs=args.epochs, validation_data=val_ds)

if __name__ == "__main__":
    main()

Overwriting custom_7/main.py


### Wrap Code in a Container Image (tensorflow)

In [100]:
%%writefile custom_7/Dockerfile

FROM tensorflow/tensorflow

RUN pip install google-cloud-bigquery==2.34.4 \
    && pip install pandas \
    && pip install protobuf==3.19.4 \
    && pip install pyarrow

COPY main.py /main.py

CMD ["python", "main.py"]

Overwriting custom_7/Dockerfile


In [ ]:
!gcloud builds submit -t $TRAIN_IMAGE_URI custom_7/.

In [ ]:
from google.cloud.aiplatform import CustomJob

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": "n1-standard-4"
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": TRAIN_IMAGE_URI,

        }
    } 
]

job = CustomJob(
    display_name = 'visitor-purchases',
    worker_pool_specs = worker_pool_specs,
    staging_bucket = STAGING_BUCKET
)
job.run()

In [ ]:
!rm -fr custom_8
!mkdir custom_8
!touch __init__.py

## Hyperparameter Tuning [Experiments]

### Create Query Files

- Different query files based on the size of dataset; 2 months vs 9 months, this helps to optimize the time to find parameters for the Neural Network.

In [ ]:
%%writefile custom_8/dataset_1.py

query = """
WITH all_visitor_stats AS (
SELECT
  fullvisitorid,
  IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid
)
# add in new features
SELECT * EXCEPT(unique_session_id) FROM (
  SELECT
      CONCAT(fullvisitorid, CAST(visitId AS STRING)) AS unique_session_id,
      # labels
      will_buy_on_return_visit,
      MAX(CAST(h.eCommerceAction.action_type AS INT64)) AS latest_ecommerce_progress,
      # behavior on the site
      IFNULL(totals.bounces, 0) AS bounces,
      IFNULL(totals.timeOnSite, 0) AS time_on_site,
      IFNULL(totals.pageviews, 0) AS pageviews,
      # where the visitor came from
      trafficSource.source,
      trafficSource.medium,
      channelGrouping,
      # mobile or desktop
      device.deviceCategory,
      # geographic
      IFNULL(geoNetwork.country, "") AS country
  FROM `data-to-insights.ecommerce.web_analytics`,
     UNNEST(hits) AS h
    JOIN all_visitor_stats USING(fullvisitorid)
  WHERE 1=1
    # only predict for new visits
    AND totals.newVisits = 1
    AND date BETWEEN '20160801' AND '20161001' # train 2 months
  GROUP BY
  unique_session_id,
  will_buy_on_return_visit,
  bounces,
  time_on_site,
  totals.pageviews,
  trafficSource.source,
  trafficSource.medium,
  channelGrouping,
  device.deviceCategory,
  country)
"""

In [ ]:
%%writefile custom_8/dataset_2.py

query = """
WITH all_visitor_stats AS (
SELECT
  fullvisitorid,
  IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid
)
# add in new features
SELECT * EXCEPT(unique_session_id) FROM (
  SELECT
      CONCAT(fullvisitorid, CAST(visitId AS STRING)) AS unique_session_id,
      # labels
      will_buy_on_return_visit,
      MAX(CAST(h.eCommerceAction.action_type AS INT64)) AS latest_ecommerce_progress,
      # behavior on the site
      IFNULL(totals.bounces, 0) AS bounces,
      IFNULL(totals.timeOnSite, 0) AS time_on_site,
      IFNULL(totals.pageviews, 0) AS pageviews,
      # where the visitor came from
      trafficSource.source,
      trafficSource.medium,
      channelGrouping,
      # mobile or desktop
      device.deviceCategory,
      # geographic
      IFNULL(geoNetwork.country, "") AS country
  FROM `data-to-insights.ecommerce.web_analytics`,
     UNNEST(hits) AS h
    JOIN all_visitor_stats USING(fullvisitorid)
  WHERE 1=1
    # only predict for new visits
    AND totals.newVisits = 1
    AND date BETWEEN '20160801' AND '20170430' # train 9 months
  GROUP BY
  unique_session_id,
  will_buy_on_return_visit,
  bounces,
  time_on_site,
  totals.pageviews,
  trafficSource.source,
  trafficSource.medium,
  channelGrouping,
  device.deviceCategory,
  country)
"""

### Training code for Hyperparameter Tuning Training and Custom Training

In [130]:
%%writefile custom_8/main.py
import os
import argparse
import warnings
import hypertune
import dataset_1, dataset_2
import numpy as np
import pandas as pd
import tensorflow as tf
from google.cloud import bigquery 
from tensorflow.keras import layers
warnings.filterwarnings('ignore')

### Bigquery Definition


def get_args():
    '''Parses args. Must include all hyperparameters you want to tune.'''
    
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--batch_size',
        default=256,
        type=int,
        help='batch size to build tf.data.Dataset')
    parser.add_argument(
        '--learning_rate',
        default=0.001,
        type=float,
        help='learning rate')
    parser.add_argument(
        '--num_neurons',
        default=32,
        type=int,
        help='number of units in the first hidden layer')
    parser.add_argument(
        '--label_column',
        default='will_buy_on_return_visit',
        type=str,
        help='The column to predict (label/target)')
    parser.add_argument(
        '--epochs',
        default=10,
        type=int,
        help='Numbber of epochs for the training; complete pass over dataset')
    parser.add_argument(
        '--hptune',
        default=False,
        type=bool,
        help='Hyperparameter tuning')
    args = parser.parse_args()
    return args

def preprocessing(query: str, batch_size: int, target_column: str):
  '''Split and Transform data into tf.Dataset, shuffles + batch'''
  
  client = bigquery.Client(project=os.environ['CLOUD_ML_PROJECT_ID'])
  df = client.query(query).to_dataframe()
  train, val, test = np.split(df.sample(frac=1), [int(0.8*len(df)), int(0.9*len(df))])


  ## Transforming data to tf.data.Dataset (Multidimmension)
  
  def df_to_dataset(dataframe, batch_size: str, shuffle=True):
    for column in dataframe.columns:
      if dataframe[column].dtype == 'Int64':
        dataframe[column] = dataframe[column].astype(np.int64)
      df = dataframe.copy()
      labels = df.pop(target_column)
      df = {key: value[:,tf.newaxis] for key, value in dataframe.items()}
      ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
      if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
      ds = ds.batch(batch_size)
      ds = ds.prefetch(batch_size)
    return ds

  ## Tensorflow Normalization

  def get_normalization_layer(name, dataset):
    
    normalizer = layers.Normalization(axis=None)
    feature_ds = dataset.map(lambda x, y: x[name])    
    normalizer.adapt(feature_ds)
    
    return normalizer

  ## Encoding for categorical data

  def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):

    if dtype == 'string':
      index = layers.StringLookup(max_tokens=max_tokens)
    else:
      index = layers.IntegerLookup(max_tokens=max_tokens)

    feature_ds = dataset.map(lambda x, y: x[name])
    index.adapt(feature_ds)
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())

    # Apply multi-hot encoding to the indices. The lambda function captures the
    # layer, so you can use them, or include them in the Keras Functional model later.
    return lambda feature: encoder(index(feature))
    
  train_ds = df_to_dataset(train, batch_size=batch_size)
  val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
  test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)
  
  all_inputs = []
  encoded_features = []

  # Numerical features.
  cat_columns = [i for i in df if df[i].dtypes == 'object' and i != target_column]
  num_columns = [i for i in df if df[i].dtypes == 'int64' and i != target_column]

  for header in num_columns:
    numeric_col = tf.keras.Input(shape=(1,), name=header)
    normalization_layer = get_normalization_layer(header, train_ds)
    encoded_numeric_col = normalization_layer(numeric_col)
    all_inputs.append(numeric_col)
    encoded_features.append(encoded_numeric_col)

  for header in cat_columns:
    categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='string')
    encoding_layer = get_category_encoding_layer(
      name=header,
      dataset=train_ds,
      dtype='string',
      max_tokens=5)
  encoded_categorical_col = encoding_layer(categorical_col)
  all_inputs.append(categorical_col)
  encoded_features.append(encoded_categorical_col)

  return all_inputs, encoded_features, train_ds, val_ds, test_ds

def create_model(all_inputs, encoded_features, nn_input: int, lr: float):
  '''Train model with TF+Keras'''
  all_features = tf.keras.layers.concatenate(encoded_features)
  x = tf.keras.layers.Dense(nn_input, activation="relu")(all_features)
  x = tf.keras.layers.Dropout(0.5)(x)
  output = tf.keras.layers.Dense(1)(x)
  
  model = tf.keras.Model(all_inputs, output)
  model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=["accuracy"])
  return model  


def main():
  args = get_args()
  if args.hptune == True:
    query = dataset_1.query
    epochs = 4
  else: 
    query = dataset_2.query
    epochs = args.epochs
  
  all_inputs, encoded_features, train_ds, val_ds, test_ds = preprocessing(query, args.batch_size, args.label_column)
  model = create_model(all_inputs, encoded_features, args.num_neurons, args.learning_rate)
  history = model.fit(train_ds, epochs=epochs, validation_data=val_ds)

  hp_metric = history.history['val_accuracy'][-1]

  hpt = hypertune.HyperTune()
  hpt.report_hyperparameter_tuning_metric(
    hyperparameter_metric_tag='accuracy',
    metric_value=hp_metric,
    global_step=args.epochs)

if __name__ == "__main__":
    main()

Overwriting custom_8/main.py


### Wrap Code with a Container Image

In [131]:
%%writefile custom_8/Dockerfile

FROM tensorflow/tensorflow

RUN pip install google-cloud-bigquery==2.34.4 \
    && pip install pandas \
    && pip install protobuf==3.19.4 \
    && pip install pyarrow \
    && pip install cloudml-hypertune

COPY * /main.py

ENTRYPOINT ["python", "main.py"]

Overwriting custom_8/Dockerfile


### Cloud Build to Create and Push Image to Google Cloud Repository

In [ ]:
!gcloud builds submit -t gcr.io/jchavezar-demo/visitor-purchase-train-ml:latest custom_8/.

### Run the Hyperparameter Tuning Training Job

In [133]:
from google.cloud import aiplatform
from google.cloud.aiplatform import hyperparameter_tuning as hpt

aiplatform.init(project='jchavezar-demo', location='us-central1', staging_bucket='gs://vtx-staging')

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": "n1-standard-4",
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": "gcr.io/jchavezar-demo/visitor-purchase-train-ml:latest",
            "command": [],
            "args": [],
        },
    }
]

custom_job = aiplatform.CustomJob(
    display_name='custom_job',
    worker_pool_specs=worker_pool_specs,
)

hpt_job = aiplatform.HyperparameterTuningJob(
    display_name='purchase-hpt',
    custom_job=custom_job,
    metric_spec={
        'loss': 'minimize',
    },
    parameter_spec={
        'learning_rate': hpt.DoubleParameterSpec(min=0.001, max=0.1, scale='log'),
        'num_neurons': hpt.IntegerParameterSpec(min=4, max=128, scale='linear'),
        'batch_size': hpt.DiscreteParameterSpec(values=[128, 256], scale='linear')
    },
    max_trial_count=128,
    parallel_trial_count=8,
    labels={'hpt_test': 'jca'},
    )

hpt_job.run()

HyperparameterTuningJob projects/569083142710/locations/us-central1/hyperparameterTuningJobs/228604685211140096 current state:
JobState.JOB_STATE_RUNNING
